# Backpropagation Practice

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
import numpy as np
np.random.seed(1)

In [3]:
class Neural_Network(object):
    def __init__(self):
        self.inputs = 3
        self.hiddenNodes = 4
        self.outputNodes = 1

        # Initlize Weights
        self.L1_weights = np.random.randn(self.inputs, self.hiddenNodes) # (3x2)
        self.L2_weights = np.random.randn(self.hiddenNodes, self.outputNodes) # (3x1)

    def feed_forward(self, X):
        # Weighted sum between inputs and hidden layer
        self.hidden_sum = np.dot(X, self.L1_weights)
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        # Weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
        # final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        return self.activated_output
        
    def sigmoid(self, s):
        return 1/(1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def backward(self, X, y, o):
        # backward propgate through the network
        self.o_error = y - o # error in output
        self.o_delta = self.o_error*self.sigmoidPrime(o) # applying derivative of sigmoid to error

        self.z2_error = self.o_delta.dot(self.L2_weights.T) # z2 error: how much our hidden layer weights contributed to output error
        self.z2_delta = self.z2_error*self.sigmoidPrime(self.activated_hidden) # applying derivative of sigmoid to z2 error

        self.L1_weights += X.T.dot(self.z2_delta) # adjusting first set (input --> hidden) weights
        self.L2_weights += self.activated_hidden.T.dot(self.o_delta) # adjusting second set (hidden --> output) weights
        
    def train (self, X, y):
        o = self.feed_forward(X)
        self.backward(X, y, o)

In [ ]:
class Neural_Network_ReLU(object):
    def __init__(self):
        self.inputs = 3
        self.hiddenNodes = 4
        self.outputNodes = 1

        # Initlize Weights
        self.L1_weights = np.random.randn(self.inputs, self.hiddenNodes) # (3x4)
        self.L2_weights = np.random.randn(self.hiddenNodes, self.outputNodes) # (4x1)

    def feed_forward(self, X):
        # Weighted sum between inputs and hidden layer
        self.hidden_sum = np.dot(X, self.L1_weights)
        print(self.hidden_sum)
        print()
        # Activations of weighted sum
        self.activated_hidden = self.ReLU(self.hidden_sum)
        # Weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
        # final activation of output
        self.activated_output = self.ReLU(self.output_sum)
        return self.activated_output
    
    def ReLU(self, s): 
        ReLU_vec = np.vectorize(lambda x: 0 if x < 0 else x)
        return ReLU_vec(s)
        
    def ReLUPrime(self, s):
        ReLUPrime_vec = np.vectorize(lambda x: 0 if x < 0 else 1)
        return ReLUPrime_vec(s)
  
    def backward(self, X, y, output):
        # backward propgate through the network
        self.output_error = y - output # error in output
        self.output_delta = self.output_error*self.ReLUPrime(output) # applying derivative of ReLU to the error

        self.z2_error = self.output_delta.dot(self.L2_weights.T) # z2 error: how much our hidden layer weights contributed to output error
        self.z2_delta = self.z2_error*self.ReLUPrime(self.activated_hidden) # applying derivative of ReLU to z2 error

        self.L1_weights += X.T.dot(self.z2_delta) # adjusting first set (input --> hidden) weights
        self.L2_weights += self.activated_hidden.T.dot(self.output_delta) # adjusting second set (hidden --> output) weights

    def train (self, X, y):
        output = self.feed_forward(X)
        self.backward(X, y, output)

In [6]:
X = np.array(([0,0,1],[0,1,1],[1,0,1],[0,1,0],[1,0,0],[1,1,1],[0,0,0]), dtype=float)
y = np.array(([0],[1],[1],[1],[1],[0],[0]), dtype=float)

NN = Neural_Network()
for i in range(1000): # trains the NN 1,000 times
    if i+1 in [1,2,3,4,5] or (i+1) % 50 == 0:
        print('+---------- EPOCH', i+1, '-----------+')
        print("Input: \n", X) 
        print("Actual Output: \n", y)    
        print("Predicted Output: \n" + str(NN.feed_forward(X))) 
        print("Loss: \n" + str(np.mean(np.square(y - NN.feed_forward(X))))) # mean sum squared loss
        print("\n")
    NN.train(X, y)

+---------- EPOCH 1 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[0.84189004]
 [0.86529986]
 [0.8489874 ]
 [0.77557764]
 [0.71782001]
 [0.87277068]
 [0.7010895 ]]
Loss: 
0.30471055257769036


+---------- EPOCH 2 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[0.80484792]
 [0.82891609]
 [0.82000855]
 [0.72666681]
 [0.68345122]
 [0.84342196]
 [0.6527522 ]]
Loss: 
0.2888295696524673


+---------- EPOCH 3 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[0.7637135 ]
 [0.78794133]
 [0.78843784]
 [0.67601178]
 [0.64915792]
 [0.81094442]
 [0.60428066]]
Loss: 
0.2748328904705085




## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [32]:
from mlxtend.data import mnist_data
import matplotlib.pyplot as plt
from mlxtend.preprocessing import shuffle_arrays_unison
from sklearn.preprocessing import StandardScaler

In [127]:
# Import data
X, y = mnist_data()

# Scale X
X /= 255

# Replace y with a vector of probabilities that the digit is what it is
y_expanded = []
for idx, num in enumerate(y):
    row = np.zeros(10)
    row[num] = 1
    y_expanded.append(row)

y = np.array(y_expanded)

In [100]:
# I'll extract 10% of the digits for train/test datasets.
X, y = shuffle_arrays_unison((X, y), random_seed=1)
X_train, y_train = X[:500], y[:500]
X_test, y_test = X[500:], y[500:]

In [131]:
# Let's make a larger NN, with 2 hidden layers of 16 each, and an output of 10 variables
# (the probability prediction for each digit)
class Neural_Network_MNIST(object):
    def __init__(self):
        self.inputs = 784
        self.L1Nodes = 16
        self.L2Nodes = 16
        self.outputNodes = 10

        # Initlize Weights
        self.L1_weights = np.random.randn(self.inputs, self.L1Nodes) # (784x16)
        self.L2_weights = np.random.randn(self.L1Nodes, self.L2Nodes) # (16x16)
        self.output_weights = np.random.randn(self.L2Nodes, self.outputNodes) # (16x10)

    def feed_forward(self, X):
        # Sum and activate flows to L1
        self.activated_L1 = self.sigmoid(np.dot(X, self.L1_weights)) 
        # Sum and activate flows to L2
        self.activated_L2 = self.sigmoid(np.dot(self.activated_L1, self.L2_weights))
        # Sum and activate flows to output
        self.activated_output = self.sigmoid(np.dot(self.activated_L2, self.output_weights))
        return self.activated_output
        
    def sigmoid(self, s):
        return 1/(1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def backward(self, X, y, output):
        ## backward propgate through the network, calculating error and delta at each layer
        # Output
        self.output_error = y - output # error in this layer
        self.output_delta = self.output_error*self.sigmoidPrime(output) # apply derivative of sigmoid to error
        
        # L2
        self.L2_error = self.output_delta.dot(self.output_weights.T) 
        self.L2_delta = self.L2_error*self.sigmoidPrime(self.activated_L2)
        
        # L1
        self.L1_error = self.L2_delta.dot(self.L2_weights.T) 
        self.L1_delta = self.L1_error*self.sigmoidPrime(self.activated_L1)
        
        
        ## Update all weights
        self.L1_weights += X.T.dot(self.L1_delta) 
        self.L2_weights += self.activated_L1.T.dot(self.L2_delta)
        self.output_weights += self.activated_L2.T.dot(self.output_delta)
        
    def train (self, X, y):
        output = self.feed_forward(X)
        self.backward(X, y, output)

In [132]:
nn2 = Neural_Network_MNIST()

for i in range(4): # trains the NN 1,000 times
    nn2.train(X_train, y_train)
#     y_pred = nn2.feed_forward(X_train)
#     loss = np.mean(np.square(y_train - nn2.feed_forward(X_train))) # mean sum squared loss
#     print(f'i: {loss}')

    print(nn2.feed_forward(X_train)[1])
    print()
# print(nn2.output_weights)

[1.22202683e-03 6.92890089e-23 2.48182092e-63 4.19831672e-20
 7.11424974e-21 5.50774901e-40 1.96195144e-71 1.35335074e-64
 1.06331026e-67 3.60133247e-65]

[1.80471573e-03 7.00660954e-23 2.58332989e-63 4.27987201e-20
 7.23873572e-21 5.70384466e-40 2.11492318e-71 1.45068092e-64
 1.14894658e-67 3.88137117e-65]

[3.17923771e-03 7.11860040e-23 2.73484683e-63 4.39856434e-20
 7.41949240e-21 5.99473844e-40 2.35317960e-71 1.60123947e-64
 1.28271374e-67 4.31743365e-65]

[8.32308492e-03 7.30786409e-23 3.00533965e-63 4.60224568e-20
 7.72856978e-21 6.50904563e-40 2.80782885e-71 1.88547050e-64
 1.53913555e-67 5.14923471e-65]



All right, this version fails to converge and the values of the weights all quickly go to basically zero.  Time to rewrite.

# Neural Net 2
With this structure, albeit with different dimensions:

![ANN diagram](https://cdn-images-1.medium.com/max/720/1*ykRnl7_YLvDmsDGWip3-kA.png)

Source: https://towardsdatascience.com/neural-networks-from-scratch-easy-vs-hard-b26ddc2e89c7

In [161]:
class NN2(object):
    def __init__(self, x, y):
        self.x = x
        neurons = 128
        self.lr = 0.5
        ip_dim = x.shape[1]
        op_dim = y.shape[1]

        self.w1 = np.random.randn(ip_dim, neurons)
        self.b1 = np.zeros((1, neurons))
        self.w2 = np.random.randn(neurons, neurons)
        self.b2 = np.zeros((1, neurons))
        self.w3 = np.random.randn(neurons, op_dim)
        self.b3 = np.zeros((1, op_dim))
        self.y = y
        
    def feedforward(self):
        z1 = np.dot(self.x, self.w1) + self.b1
        self.a1 = sigmoid(z1)
        z2 = np.dot(self.a1, self.w2) + self.b2
        self.a2 = sigmoid(z2)
        z3 = np.dot(self.a2, self.w3) + self.b3
        self.a3 = softmax(z3) # The original called for sigmoid here,
                              # which I'm pretty sure is a typo.

    def backprop(self):
        loss = error(self.a3, self.y)
#         print('Error :', loss)
        a3_delta = cross_entropy(self.a3, self.y) # w3
        z2_delta = np.dot(a3_delta, self.w3.T)
        a2_delta = z2_delta * sigmoid_derv(self.a2) # w2
        z1_delta = np.dot(a2_delta, self.w2.T)
        a1_delta = z1_delta * sigmoid_derv(self.a1) # w1

        self.w3 -= self.lr * np.dot(self.a2.T, a3_delta)
        self.b3 -= self.lr * np.sum(a3_delta, axis=0, keepdims=True)
        self.w2 -= self.lr * np.dot(self.a1.T, a2_delta)
        self.b2 -= self.lr * np.sum(a2_delta, axis=0)
        self.w1 -= self.lr * np.dot(self.x.T, a1_delta)
        self.b1 -= self.lr * np.sum(a1_delta, axis=0)

    def predict(self, data):
        self.x = data
        self.feedforward()
        return self.a3.argmax()
    
def sigmoid(s):
    return 1/(1 + np.exp(-s))

def sigmoid_derv(s):
    return s * (1 - s)

def softmax(s):
    exps = np.exp(s - np.max(s, axis=1, keepdims=True))
    return exps/np.sum(exps, axis=1, keepdims=True)

def cross_entropy(pred, real):
    n_samples = real.shape[0]
    res = pred - real
    return res/n_samples

def error(pred, real):
    n_samples = real.shape[0]
    logp = - np.log(pred[np.arange(n_samples), real.argmax(axis=1)])
    loss = np.sum(logp)/n_samples
    return loss

def get_acc(x, y):
    acc = 0
    for xx,yy in zip(x, y):
        s = model.predict(xx)
        if s == np.argmax(yy):
            acc +=1
    return acc/len(x)*100

Same inputs as before:

In [144]:
# Import data
X, y = mnist_data()

# Scale X
X /= 255

# Replace y with a vector of probabilities that the digit is what it is
y_expanded = []
for idx, num in enumerate(y):
    row = np.zeros(10)
    row[num] = 1
    y_expanded.append(row)

y = np.array(y_expanded)

In [100]:
# I'll extract 10% of the digits for train/test datasets.
X, y = shuffle_arrays_unison((X, y), random_seed=1)
X_train, y_train = X[:500], y[:500]
X_test, y_test = X[500:], y[500:]

Drumroll...

In [172]:
model = NN2(X_train, y_train)

epochs = 1500
for x in range(epochs):
    model.feedforward()
    model.backprop()

print("Train accuracy :", get_acc(X_train, y_train))
print("Test accuracy :", get_acc(X_test, y_test))

Train accuracy : 100.0
Test accuracy : 51.800000000000004


/anaconda3/envs/sandbox/lib/python3.6/site-packages/ipykernel_launcher.py:48: RuntimeWarning: overflow encountered in exp


## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?